<a href="https://colab.research.google.com/github/JasonLimJS/Grab-AI-for-SEA/blob/master/Ensemble_of_2_kNN_Models_(Weights_Tuning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.externals import joblib
import pickle


drive.mount('drive')

In [0]:
raw= pd.read_csv('drive/My Drive/raw.csv',index_col=0) #this file is created and saved in Model Development Doc.ipynb
all_lag_1d= pd.read_csv('drive/My Drive/all_lag_1d.csv',index_col=0) #this file is created and saved in Model Development Doc.ipynb

X= all_lag_1d.drop(['geohash6','day','demand'],axis=1)
arranged_col= ['lat','long','time_min','demand_lag_1']
X= X[arranged_col].values
y= all_lag_1d.demand.values
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state= 7, \
                                                   test_size=0.2,stratify=X[:,2])

X_test_1= X_test[:,:-1]

mean_vect= np.array([ -5.34803514,  90.76456439, 611.01511076])
sd_vect= np.array([5.74593198e-02, 1.02564882e-01, 3.91570353e+02])

X_test_scaled1= (X_test_1- mean_vect)/sd_vect


In [0]:
norm_mean= np.array([-5.34740478e+00,9.07638854e+01,6.10487686e+02,1.05574095e-01])
norm_std= np.array([5.65621040e-02,1.02704766e-01,3.92263089e+02,1.59564436e-01])

pca_mean= np.array([ 6.14163081e-15 ,1.66324146e-14 ,2.60231469e-16,-2.49625107e-14])
pca_comp= np.array([[ 0.64256504 ,0.72425171, -0.21969293, -0.11960199],
                    [-0.21087891,  0.07957761, -0.71483013,  0.66197838],
                    [ 0.51445685, -0.2009283,   0.46599514,  0.69123838]])

td0= (X_test- norm_mean)/norm_std
td1 = td0 - pca_mean
X_test_scaled2 = td1.dot(pca_comp.T)

In [0]:
model1= joblib.load('drive/My Drive/kNN_Model1.pkl')
model2= joblib.load('drive/My Drive/whole_lag_1d_kNN.pkl')
weight_vect= np.array(range(0,110,10))*0.01

for weight in weight_vect:
  weight_1= weight
  weight_2= 1 - weight_1
  
  X= all_lag_1d.drop(['geohash6','day','demand'],axis=1)
  arranged_col= ['lat','long','time_min','demand_lag_1']
  X= X[arranged_col].values
  y= all_lag_1d.demand.values
  X_train, X_test, y_train, y_test= train_test_split(X, y, random_state= 7, \
                                                   test_size=0.2,stratify=X[:,2])
  X_test_1= X_test[:,:-1]

  mean_vect= np.array([ -5.34803514,  90.76456439, 611.01511076])
  sd_vect= np.array([5.74593198e-02, 1.02564882e-01, 3.91570353e+02])

  X_test_scaled1= (X_test_1- mean_vect)/sd_vect
  
  norm_mean= np.array([-5.34740478e+00,9.07638854e+01,6.10487686e+02,1.05574095e-01])
  norm_std= np.array([5.65621040e-02,1.02704766e-01,3.92263089e+02,1.59564436e-01])
  pca_mean= np.array([ 6.14163081e-15 ,1.66324146e-14 ,2.60231469e-16,-2.49625107e-14])
  pca_comp= np.array([[ 0.64256504 ,0.72425171, -0.21969293, -0.11960199],
                    [-0.21087891,  0.07957761, -0.71483013,  0.66197838],
                    [ 0.51445685, -0.2009283,   0.46599514,  0.69123838]])

  td0= (X_test- norm_mean)/norm_std
  td1 = td0 - pca_mean
  X_test_scaled2 = td1.dot(pca_comp.T)

  y_pred1= model1.predict(X_test_scaled1)
  y_pred2= model2.predict(X_test_scaled2)
  y_pred_final= weight_1*y_pred1 + weight_2*y_pred2

  rmse= MSE(y_test,y_pred_final)**0.5

  print('weight_1: ' + str(weight_1) + ' ,weight_2: ' + str(weight_2) + ', rmse: ' + str(rmse))


Tuning result:

weight_1: 0.0 ,    weight_2: 1.0,    rmse: 0.06726763281100871

weight_1: 0.1 ,    weight_2: 0.9,    rmse: 0.06446941562116118

weight_1: 0.2 ,    weight_2: 0.8,    rmse: 0.062109990665184585

weight_1: 0.3 ,    weight_2: 0.7,    rmse: 0.06024093785527065

weight_1: 0.4 ,    weight_2: 0.6,    rmse: 0.05890895102014478

weight_1: 0.5 ,    weight_2: 0.5,    rmse: 0.05815094745799957

weight_1: 0.6 ,    weight_2: 0.4,    rmse: 0.05798943984545983

weight_1: 0.7     ,weight_2: 0.3     rmse: 0.05842937480647346

weight_1: 0.8 ,    weight_2: 0.2,    rmse: 0.059457403326402314

weight_1: 0.9 ,    weight_2: 0.1,    rmse: 0.06104382068415746

weight_1: 1.0 ,    weight_2: 0.0,    rmse: 0.06314655604606347

Optimum weights:

Model 1 (weight_1): 0.6

Model 2(weight_2): 0.4